In [33]:
from gurobipy import Model, GRB
import pandas as pd
from code_map import old_meters, new_markets, Inputs
import numpy as np
import matplotlib.pyplot as plt
import calendar 
from datetime import datetime
import pytz
import openpyxl
from itertools import combinations, product


In [2]:
i_v = Inputs.one_day
H = pd.date_range(start=pd.Timestamp(year= i_v.year, month= i_v.start_month, day = i_v.start_day, hour = i_v.start_hour), 
                              end= pd.Timestamp(year = i_v.year, month = i_v.end_month, day = i_v.end_day, hour = i_v.end_hour), freq="H", tz = "Europe/Oslo")

power_meter_dict = old_meters.power_meters

# Define the sets
L = list(power_meter_dict.values())  # List of PowerMeter objects
M = new_markets.all_market_list  # List of ReserveMarket objects

In [3]:
print(f"Amount of meters : {len(M)}")
print(f"Amount of markets : {len(L)}")
print(f"Amount of hours : {len(H)}")

Amount of meters : 62
Amount of markets : 2202
Amount of hours : 24


In [4]:
#pp = np.array([[ len(market.price_data.loc[market.price_data["Time(Local)"] == hour].values) for market in M] for hour in H])

In [5]:
#pp[3][1]

In [6]:
"""for i in range(pp.shape[0]):
    for j in range(pp.shape[1]):
        if pp[i,j] != 1:
            print(i, j, pp[i,j])
            """

'for i in range(pp.shape[0]):\n    for j in range(pp.shape[1]):\n        if pp[i,j] != 1:\n            print(i, j, pp[i,j])\n            '

In [7]:
#M[15].price_data.loc[M[15].price_data["Time(Local)"] == H[7]].values

In [8]:
#M[15].price_data

In [9]:
#M[20].name

In [18]:
F_h_l = np.array([[load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for load in L] for hour in H]) # set of flex volumes for meters

R_h_l = np.array([[load.response_time for load in L]] * len(H)) # set of response times for meters

P_h_m = np.array([[market.price_data.loc[market.price_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of prices for markets
Vp_h_m = np.array([[market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1] for market in M] for hour in H]) # set of volumes for markets

Vm_m = [market.min_volume for market in M] # set of min values for markets
R_m = [market.response_time for market in M] # set of response times for markets


In [19]:

total_flex = np.sum(F_h_l)
flex_pr_hour = total_flex/len(H)

print(f"Total flex volume [MW]: {total_flex}")
print(f"Flex pr hour [MW]: {flex_pr_hour}")

Total flex volume [MW]: 195.31786599999995
Flex pr hour [MW]: 8.138244416666664


In [20]:
mean_price = np.mean(P_h_m)
print(f"Mean price [EUR/MWH]: {mean_price}")

Mean price [EUR/MWH]: 15.389086021505376


In [21]:
len(F_h_l) == len(R_h_l) == len(P_h_m) == len(Vp_h_m)

True

In [22]:
len(Vm_m) == len(R_m)

True

In [23]:
I = {}
for l in range(len(L)):
    for m in range(len(M)): 
        for h in range(len(H)):
            I[h,l,m] = F_h_l[h,l] * P_h_m[h,m] 

In [24]:
"""else:
        continue"""

compatible_list = []
for h, hour in enumerate(H):
    hour_list = []
    for l, asset in enumerate(L):
        asset_list = []
        for m, market in enumerate(M):
            if asset.direction == "up":
                if market.direction == "up":
                    if market.area == asset.area or market.area == "all":
                        asset_list.append(m)
            elif asset.direction == "down":
                if market.area == asset.area or market.area == "all":
                    if market.direction == "down":
                        asset_list.append(m)
                
            elif asset.direction == "both":
                if market.area == asset.area  or market.area == "all":
                    asset_list.append(m)
        hour_list.append(asset_list)
    compatible_list.append(hour_list)

                
    
    

In [25]:
# Create a new model
test_model = Model("AssetToMarket")

# Create decision variables
x = {}
d = {}
y = {}
for h in range(len(H)):
    for l in range(len(L)):
        for m in range(len(M)):
            # asset i is connected to market j at hour h
            x[h, l, m] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"x_{h}_{l}_{m}")

            d[h,l,m] = 1 if m in compatible_list[h][l] else 0
            
            # adding the constraint
            test_model.addConstr(x[h,l,m] <= d[h,l,m])
    for m in range(len(M)):
        # market j is active at hour h
        y[h, m] = test_model.addVar(lb = 0, ub = 1, vtype=GRB.BINARY, name=f"y_{h}_{m}")


# Set objective
test_model.setObjective(
    # maxmize income 
    sum(x[h, l, m] *I[h, l, m] for l in range(len(L)) for m in range(len(M)) for h in range(len(H))), GRB.MAXIMIZE)

# Add constraints
for h in range(len(H)):
    for l in range(len(L)):
        # Each asset can only be connected to one market at a time
        test_model.addConstr(sum(x[h, l, m] for m in range(len(M))) <= 1, f"single_market_for_asset_at_hour_{h}_nr.{l}")

    for m in range(len(M)):
        # Connect the binary variables by using big M
        test_model.addConstr((sum(x[h, l, m] for l in range(len(L)))) <= len(L) * y[h, m], f"asset_connection_for_hour_{h}_market_{m}")
        
        # Min volume constraint
        test_model.addConstr(sum(x[h, l, m] * F_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes

        # Weighted average constraint for response time
        total_response_time = sum(x[h, l, m] * R_h_l[h,l] for l in range(len(L)))
        total_assigned_assets = sum(x[h, l, m] for l in range(len(L)))
        test_model.addConstr(total_response_time <= R_m[m] * total_assigned_assets, f"avg_response_time_for_hour_{h}_market_{m}")

        # Max volume constraint
        test_model.addConstr(sum(x[h, l, m] * F_h_l[h,l] for l in range(len(L))) <=  Vp_h_m[h,m]  * y[h,m], f"max_volume_for_hour_{h}_market_{m}")

# Solve the model
test_model.optimize()
    
if test_model.status == GRB.Status.INFEASIBLE:
    test_model.computeIIS()


Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-18
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3335376 rows, 3278064 columns and 19181204 nonzeros
Model fingerprint: 0x2cdc46a8
Variable types: 0 continuous, 3278064 integer (3278064 binary)
Coefficient statistics:
  Matrix range     [5e-07, 2e+03]
  Objective range  [1e-06, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 3296904 rows and 3131089 columns (presolve time = 5s) ...
Presolve removed 3306511 rows and 3159286 columns (presolve time = 10s) ...
Presolve removed 3334504 rows and 3273933 columns (presolve time = 17s) ...
Presolve removed 3334504 rows and 3273933 columns
Presolve time: 17.57s
Presolved: 872 rows, 4131 columns, 18887 nonzeros
Found heuristic solut

In [26]:
def get_market_count_dict(x):
    data = []

    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            for m, market in enumerate(M):
                if x[h, l, m].X > 0.5:
                    data.append([hour, load.meter_id, market.name])

    df = pd.DataFrame(data, columns=["Hour", "Asset Meter ID", "Market"])
    #print(df)


    market_count_dict = {}
    for hour in H:
        hour_df = df.loc[(df["Hour"] == hour)]
        # get the number of assets in each market
        market_count = hour_df.groupby(["Market", "Hour"]).agg("count").reset_index().rename(columns={"Asset Meter ID": "Asset Count"})
        market_count_dict[hour] = market_count
    return market_count_dict

In [27]:
market_count_dict = get_market_count_dict(x)


In [28]:
def test_solution_validity(x, y, L, M, H):
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            # Each asset can only be connected to one market at a time
            assert sum(x[h, l, m].X for m in range(len(M))) <= 1, f"Asset {l} connected to multiple markets at hour {h}"
            for m, market in enumerate(M):
                # Directionality constraints
                if load.direction == "up" and market.direction == "down":
                    assert x[h, l, m].X == 0, f"Up-direction asset {l} connected to down-direction market {m} at hour {h}"
                elif load.direction == "down" and market.direction == "up":
                    assert x[h, l, m].X == 0, f"Down-direction asset {l} connected to up-direction market {m} at hour {h}"
                elif market.direction == "both" and load.direction != "both":
                    assert x[h, l, m].X == 0, f"Asset {l} with specific direction connected to both-direction market {m} at hour {h}"
                elif market.area != load.area:
                    assert x[h, l, m].X == 0, f"Asset {l} in area {load.area} connected to market {m} in area {market.area} at hour {h}"
        for m, market in enumerate(M):
            # Connect the binary variables by using big M
            assert sum(x[h, l, m].X for l in range(len(L))) <= len(L) * y[h, m].X, f"More than allowed assets connected at hour {h} to market {m}"

            # Min volume constraint
            total_flex_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))
            assert total_flex_volume >= market.min_volume * y[h, m].X, f"Minimum volume constraint violated at hour {h} for market {m}"

            # test_model.addConstr(sum(x[h, l, m] * F_h_l[h,l] for l in range(len((L)))) >= Vm_m[m] * y[h, m], f"min_volume_for_hour_{h}_market_{m}") # denne må sjekkes
            # F_h_l = np.array([[meter.flex_volume["value"].loc[meter.flex_volume["Time(Local)"] == hour].values[0] for meter in L] for hour in H]) # set of flex volumes for meters


            # Weighted average constraint for response time
            total_response_time = sum(x[h, l, m].X * load.response_time for l, load in enumerate(L))
            total_assigned_assets = sum(x[h, l, m].X for l in range(len(L)))
            assert total_response_time <= market.response_time * total_assigned_assets, f"Average response time constraint violated at hour {h} for market {m}"

            # Max volume constraint
            total_max_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))
            market_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
            assert total_max_volume <= market_max_volume * y[h,m].X, f"Maximum volume constraint violated at hour {h} for market {m}"





In [29]:
test_solution_validity(x, y, L, M, H)


In [30]:
for h, hour in enumerate(H):
    for l, load in enumerate(L):
        if load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] != F_h_l[h,l]:
            print("ERROR")
            print(load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0], F_h_l[h,l])


In [31]:
for m, market in enumerate(M):
    if Vm_m[m] != market.min_volume:
        print("ERROR")
        print(Vm_m[m], market.min_volume)

In [32]:
F_h_l = np.array([[load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for load in L] for hour in H]) # set of flex volumes for meters


In [ ]:
all(F_h_l[0,:] == [load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == H[0]].values[0] for l, load in enumerate(L)])

In [ ]:
#h = 0
m = 26
for hour in range(len(H)):
    total_flex_volume_test = [x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == H[h]].values[0] for l, load in enumerate(L)]
    total_flex_volume_mod = [x[h, l, m].X * F_h_l[h,l] for l, load in enumerate(L)]
    test_sum = sum(total_flex_volume_test)
    mod_sum = sum(total_flex_volume_mod)
    print(f" the sums are eaqual : {test_sum == mod_sum}")
    #print(f"all the values are eaqual : {all(total_flex_volume_test == total_flex_volume_mod)}")


"""print(f"The flex volume list for the test for market {m} at hour {h} is {total_flex_volume_test}")
print(f"The flex volume list for the model for market {m} at hour {h} is {total_flex_volume_mod}")
print(f"The total flex volume in the test for market {m} at hour {h} is {test_sum}")
print(f"The total flex volume in the model for market {m} at hour {h} is {mod_sum}")"""
    

In [ ]:
market_count_dict[H[23]]

In [ ]:
M[26].name